In [1]:
import import_ipynb
import polars as pl
import coingecko_api as ca

In [ ]:
def clean_xrp_bsv(scraped: dict) -> dict:
    """Calculate average_transaction_fees for XRP/BSV dataframes. Merge API and Messari dataframes for those projects"""
    # Merge XRP and BSV dataframes
    for name in ['xrp', 'bsv']:
        scraped[name] = scraped[f'{name}_bit'].join(scraped[f'{name}_mes'], on='date', how='inner')
        # Calculate average_transaction_fees column
        scraped[name] = scraped[name].with_columns((pl.col("total_fees") / pl.col("transactions_count")).alias("average_transaction_fees"))
        # Drop total_fees column
        scraped[name] = scraped[name].drop('total_fees')
    return scraped

In [ ]:
def merge_api_scraped(filled_api: dict, filled_scraped: dict, names: list) -> dict:
    """Merge API and scraped data into a single dictionary of dataframes"""
    # Fill missing dates in Messari data with null values
    for name in ['xrp_mes', 'bsv_mes', 'xlm']:
        filled_scraped[name] = ca.fill_date(filled_scraped[name])

    # Clean XRP and BSV dataframes
    clean_xrp_bsv(filled_scraped)

    # Merge API and scraped data
    combined = {}
    for name in names:
        combined[name] = filled_api[name].join(filled_scraped[name], on='date', how='inner')
    
    return combined